# **CSCE 5218 / CSCE 4930 Deep Learning**

# **HW1a The Perceptron** (20 pt)


In [27]:
# Get the datasets
!curl.exe --output train.dat http://huang.eng.unt.edu/CSCE-5218/train.dat
!curl.exe --output test.dat http://huang.eng.unt.edu/CSCE-5218/test.dat


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 11244  100 11244    0     0   417k      0 --:--:-- --:--:-- --:--:--  477k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2844  100  2844    0     0   110k      0 --:--:-- --:--:-- --:--:--  126k


In [28]:
# Take a peek at the datasets
!more train.dat

A1      A2      A3      A4      A5      A6      A7      A8      A9      A10     A11     A12     A13     
1       1       0       0       0       0       0       0       1       1       0       0       1       0
0       0       1       1       0       1       1       0       0       0       0       0       1       0
0       1       0       1       1       0       1       0       1       1       1       0       1       1
0       0       1       0       0       1       0       1       0       1       1       1       1       0
0       1       0       0       0       0       0       1       1       1       1       1       1       0
0       1       1       1       0       0       0       1       0       1       1       0       1       1
0       1       1       0       0       0       1       0       0       0       0       0       1       0
0       0       0       1       1       0       1       1       1       0       0       0       1       0
0       0       0       0       0       0      

In [29]:
!more test.dat

A1      A2      A3      A4      A5      A6      A7      A8      A9      A10     A11     A12     A13
1       1       1       1       0       0       1       1       0       0       0       1       1       0
0       0       0       1       0       0       1       1       0       1       0       0       1       0
0       1       1       1       0       1       1       1       1       0       0       0       1       0
0       1       1       0       1       0       1       1       1       0       1       0       1       0
0       1       0       0       0       1       0       1       0       1       0       0       1       0
0       1       1       0       0       1       1       1       1       1       1       0       1       0
0       1       1       1       0       0       1       1       0       0       0       1       1       0
0       1       0       0       1       0       0       1       1       0       1       1       1       0
1       1       1       1       0       0       1   

### Build the Perceptron Model

You will need to complete some of the function definitions below.  DO NOT import any other libraries to complete this. 

In [30]:
import math
import itertools
import re


# Corpus reader, all columns but the last one are coordinates;
#   the last column is the label
def read_data(file_name):
    f = open(file_name, 'r')

    data = []
    # Discard header line
    f.readline()
    for instance in f.readlines():
        if not re.search('\t', instance): continue
        instance = list(map(int, instance.strip().split('\t')))
        # Add a dummy input so that w0 becomes the bias
        instance = [-1] + instance
        data += [instance]
    return data


def dot_product(array1, array2):
    result = sum(a * b for a, b in zip(array1, array2))
    return result 


def sigmoid(x):
    sigmoid_x = 1 / (1 + math.exp(-x))
    return sigmoid_x

def output(weight, instance):
    dot_result = dot_product(weight, instance)
    output_result = sigmoid(dot_result)
    return output_result

def predict(weights, instance):
    model_output = output(weights, instance)
    prediction = 1 if model_output >= 0.5 else 0
    return prediction

def get_accuracy(weights, instances):
    # You do not to write code like this, but get used to it
    correct = sum([1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances])
    return correct * 100 / len(instances)

def train_perceptron(instances, lr, epochs):

    # weights initialization. 
    # We are creating a weight column with the value of 0 which will be updated later while training the model.
    weights = [0] * (len(instances[0])-1)

    for _ in range(epochs):
        for instance in instances:
            # Calculating input, output and error.
            in_value = dot_product(weights, instance)
            output = sigmoid(in_value)
            error = instance[-1] - output

            # weights updation
            # updating weights using learning rate (lr), error, output, and input features instance. 
            for i in range(0, len(weights)):
                weights[i] += lr * error * output * (1-output) * instance[i]

    return weights

## Run it

In [31]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0


## Questions

Answer the following questions. Include your implementation and the output for each question.



### Question 1

In `train_perceptron(instances, lr, epochs)`, we have the following code:
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```

Why don't we have the following code snippet instead?
```
output = predict(weights, instance)
error = instance[-1] - output
```

#### Answer:
In the code above, the 'dot_product' function calculates the linear combination of weights and inputs, and the sigmoid function transforms this linear output into a value between 0 and 1. The sigmoid activation function introduces non-linearity and ensures that the output is in a valid probability range. The error is then calculated as the difference between the actual label (instance[-1]) and the transformed output. This error is used in the perceptron learning rule to adjust the weights during training.

In the provided code, using the predict function directly results in a binary prediction (0 or 1) based on a threshold. However, this approach comes at the expense of losing the continuous-valued output obtained by applying the sigmoid function, which is essential for calculating the error and updating the weights during the training process.

### Question 2
Train the perceptron with the following hyperparameters and calculate the accuracy with the test dataset.

```
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]              # number of epochs
lr = [0.005, 0.01, 0.05]              # learning rate
```

TODO: Write your code below and include the output at the end of each training loop (NOT AFTER EACH EPOCH)
of your code.The output should look like the following:
```
# tr:  20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
[and so on for all the combinations]
```
You will get different results with different hyperparameters.

#### TODO Add your answer here (code and output in the format above) 


In [32]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]     # number of epochs
lr_array = [0.005, 0.01, 0.05]        # learning rate

for lr in lr_array:
  for tr_size in tr_percent:
    for epochs in num_epochs:
      size =  round(len(instances_tr)*tr_size/100)
      pre_instances = instances_tr[0:size]
      weights = train_perceptron(pre_instances, lr, epochs)
      accuracy = get_accuracy(weights, instances_te)
    print(f"#tr: {len(pre_instances):0}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
            f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 200, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 74.0
#tr: 300, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 78.0
#tr: 400, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 77.0
#tr: 20, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 78.0
#tr: 300, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 80.0
#tr: 400, epochs: 100, learning rate: 0.010; Accuracy (test, 100 instances): 80.0
#tr: 20, epochs: 100

### Question 3
Write a couple paragraphs interpreting the results with all the combinations of hyperparameters. Drawing a plot will probably help you make a point. In particular, answer the following:
- A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?


- B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?

   ```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```
- C. Can you get higher accuracy with additional hyperparameters (higher than `80.0`)? 

- D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

#### TODO: Add your answer here (code and text)



Answer A: No, training dataset with all the data doesn't promise highest accuracy. Accuracy depends on the quality of the data, model architecture and hyperparameters of the model.

Answer B: As we answered the above question, here we increased the dataset but other parameters of the model are same. Accuracy not only depends on the size of the data, it also depends on the quality and hyperparameter tuning.

Answer C: Increasing accuracy higher than 80 requires additional experiments regarding tr and other parameters such as epochs, learning rate. There are several strategies to do that such as Grid Search.

Answer D: No, it is not always a sensible idea to train model for more epochs because after some epochs, accuracy will stop increasing and there are very high chances of overfitting which will decrease validation accuracy. It is also very time and resource consuming and to prevent that, we use early stopping method which will stop training model once the accuracy stops increasing for the given epochs.